# Notebook Imports

In [233]:
import pandas as pd
import numpy as np

# Constants

In [307]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

## Read and Load Features from .txt Files into NumPy Array

In [235]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int) 

In [236]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [237]:
sparse_train_data[:5] # first 5 rows

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 15,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1]])

In [238]:
sparse_train_data[-5:] # last 5 rows

array([[5795, 1539,    0,    2],
       [5795, 1903,    0,    1],
       [5795, 1915,    0,    2],
       [5795, 2127,    0,    1],
       [5795, 2279,    0,    1]])

In [239]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 265427
Nr of rows in test file 110522


In [240]:
print('Nr of emails in training file', np.unique(sparse_train_data[:, 0]).size) # ':' to select all rows, 0 to select first column

Nr of emails in training file 4015


In [241]:
print('Nr of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Nr of emails in test file 1724


### How to create an Empty DataFrame

In [242]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE)) # all the vocab words are in all columns
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [243]:
len(column_names)

2502

In [244]:
index_names = np.unique(sparse_train_data[:, 0]) # all the emails are in all rows
len(index_names)

4015

In [245]:
%%time
full_train_data = pd.DataFrame(index=index_names, columns=column_names, dtype=int)
full_train_data.fillna(value=0, inplace=True)

Wall time: 220 ms


In [246]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [247]:
sparse_train_data[0:10]

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 15,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1],
       [ 0, 29,  1,  1],
       [ 0, 52,  1,  1],
       [ 0, 60,  1,  1],
       [ 0, 64,  1,  1],
       [ 0, 65,  1,  1]])

In [248]:
sparse_train_data.shape[0]

265427

### Create a Full Matrix from a Sparse Matrix

In [249]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]  # doc_nr is index of sparse matrix
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix
    

#### Creating full matrix from sparse matrix for training data

In [250]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 7.62 s


In [251]:
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,2,2,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5791,0,2,0,1,1,0,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0


### Training the Naive Bayes Mode
### $$ P( spam | word ) = \frac {P( word | spam ) P( spam )} {P( word )} $$
### $$ P( spam | word ) = \frac { \text {occurence of the spam word in spam / no. of spam words} \times \text {probability of e-mails being spam} } {\text {occurence of the spam word in all e-mail}} $$

### 1. Calculating the Probability of Spam

In [254]:
full_train_data.CATEGORY.size # total messages

4015

In [255]:
full_train_data.CATEGORY.sum() # sum all 0 and 1, 1 is spam. so total spam

1250

In [257]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.31133250311332505


### 2. Calculating Total Number of Words / Tokens

In [323]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY'] # the dataframe without CATEGORY
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [260]:
email_lengths = full_train_features.sum(axis=1) # sums all columns(axis=1)
email_lengths.shape

(4015,)

In [267]:
email_lengths[:5] # top five

DOC_ID
0     50
1     76
2     87
3     76
4    136
dtype: int64

In [264]:
total_wc = email_lengths.sum() # total words
total_wc

445891

#### Number of Tokens in Spam & Ham Emails

Challenge Create a subset of the email_lengths series that only contains the spam messages. Call the subset spam_lengths. Then count the total number of words that occur in spam emails.

Do the same for the non-spam emails. Create a subset called ham_lengths. Then count the total number of words that occur in the ham emails.

In [271]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1] # how many spam emails
spam_lengths.shape

(1250,)

In [270]:
spam_wc = spam_lengths.sum() # how many words in spam emails
spam_wc

195645

In [272]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0] # ham emails
ham_lengths.shape

(2765,)

In [273]:
nonspam_wc = ham_lengths.sum() # how many words in ham emails
nonspam_wc

250246

In [274]:
spam_wc + nonspam_wc - total_wc # should be 0

0

In [275]:
print('Average nr of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 157
Average nr of words in ham emails 90.505


#### Summing the Tokens Occuring in Spam

In [278]:
full_train_features.shape

(4015, 2500)

In [280]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1] # create subset with only spams using loc
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [282]:
train_spam_tokens.shape # too make sure its right 1250

(1250, 2500)

In [285]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 # sum all tokens on each column(asix=0) & adding one so words that don't
                                                        # occur don't have value of 0(Laplace smoothing)
summed_spam_tokens.shape # pandas series

(2500,)

In [287]:
summed_spam_tokens # word no. 0 occurs 1793 times across all spam messages

0       1793
1        968
2       1355
3       2099
4       1348
        ... 
2495      19
2496       2
2497      23
2498      10
2499       4
Length: 2500, dtype: int64

#### Summing the Tokens Occuring in Ham

In [288]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [289]:
summed_ham_tokens.shape

(2500,)

In [291]:
summed_ham_tokens # word no. 0 occurs 5273 times across all spam messages

0       5273
1       2499
2       2038
3        944
4       1595
        ... 
2495      13
2496      19
2497       1
2498      16
2499      35
Length: 2500, dtype: int64

In [293]:
train_ham_tokens[0].sum() + 1 # make sure the value is right(5273)

5273

### 3. P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [296]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE) # how many times one spam word occurs / total spam words
                                                                # added 2500 words(VOCAB_SIZE) cz we added +1 for each 2500's
prob_tokens_spam

0       0.009049
1       0.004885
2       0.006838
3       0.010593
4       0.006803
          ...   
2495    0.000096
2496    0.000010
2497    0.000116
2498    0.000050
2499    0.000020
Length: 2500, dtype: float64

In [298]:
prob_tokens_spam.sum() # if 1 then everything ties up

1.0

#### P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [311]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)# how many times one ham word occurs / total ham words
                                                                # added 2500 words(VOCAB_SIZE) cz we added +1 for each 2500's
prob_tokens_spam
prob_tokens_nonspam.sum()                                                     

1.0

In [300]:
prob_tokens_all.sum()

1.0

#### P(Token) - Probability that Token Occurs

In [303]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc # how many times one word occurs / total words ham & spam

In [304]:
prob_tokens_all.sum()

1.0

### Save the Trained Model

In [312]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

### Prepare Test Data

#### Creating full matrix from sparse matrix for testing data

In [326]:
sparse_test_data.shape

(110522, 4)

In [314]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 3.71 s


In [321]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY'] # dataframe without CATEGORY(feature)
y_test = full_test_data.CATEGORY # dataframe with only CATEGORY(target)

#### Saving target and feature of testing files as txt

In [324]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)